en el modulo basepc hemos creado un objeto para limpiar los datas mas rapidamente. lo utilizamos a continuacios

In [ ]:
from basepc import *

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
import xgboost
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
train_set = pd.read_csv('./data/train.csv').drop(['laptop_ID'], axis= 1)
test_set = pd.read_csv('./data/test.csv').drop(['laptop_ID'], axis= 1)

In [ ]:
limpiador_train = Cleaner(train_set, target= 'Price_euros')

In [ ]:
limpiador_train.clean_all()

In [ ]:
limpiador_test = Cleaner(test_set, test= True)

In [ ]:
limpiador_test.clean_all()

el objeto tiene mas funciones, como representar la cardinalidad, quitar el target y alguno mas. 

de momento vamos a hacer los splits

In [ ]:
train = limpiador_train.get_cleaned_train()
test = limpiador_test.get_cleaned_train()

In [ ]:
train_set, validation_set = train_test_split(train, test_size= 0.2, random_state= 42)

In [ ]:
x_train = train_set.drop(['Price_euros', 'id'], axis= 1)
y_train = train_set['Price_euros']

x_validation = validation_set.drop(['Price_euros', 'id'], axis= 1)
y_validation = validation_set['Price_euros']

voy a normalizar el target para que el modelo tire mejor

In [ ]:
sns.histplot(y_train)
plt.show()

In [ ]:
y_train = np.log10(y_train)

sns.histplot(y_train)
plt.show()

## vamos a probar modelos

catboost

In [ ]:
model_catboost = CatBoostRegressor(random_state= 42)

In [ ]:
model_catboost.fit(x_train, y_train, verbose= 0)

y_pred = model_catboost.predict(x_validation)
y_pred = np.power(10, y_pred) # devolvemos los valores reales a precio

mean_absolute_error(y_validation, y_pred)

In [ ]:
# vamos a tirar un test aqui para subirlo a kaggle

y_pred = model_catboost.predict(test.drop(['id'], axis= 1))
y_pred = np.power(10, y_pred) # devolvemos los valores reales a precio

In [ ]:
test['Price_euros'] = y_pred
submision = test[['id', 'Price_euros']]
submision

In [ ]:
submision.to_csv('submission.csv', index= False)

xgboost

In [ ]:
xgboost_model = xgboost.XGBRegressor()

xgboost_model.fit(x_train, y_train)

y_pred = xgboost_model.predict(x_validation)
y_pred = np.power(10, y_pred) # devolvemos los valores reales a precio

mean_absolute_error(y_validation, y_pred)

linear regression

In [ ]:
lineal_model = LinearRegression()

lineal_model.fit(x_train, y_train)
y_pred = lineal_model.predict(x_validation)
y_pred = np.power(10, y_pred) # devolvemos los valores reales a precio

mean_absolute_error(y_validation, y_pred)

gradient regressor

In [ ]:
gradient_model = GradientBoostingRegressor(random_state= 42)

gradient_model.fit(x_train, y_train)

y_pred = gradient_model.predict(x_validation)
y_pred = np.power(10, y_pred) # devolvemos los valores reales a precio

mean_absolute_error(y_validation, y_pred)

random forest

In [ ]:
forest = RandomForestRegressor(random_state= 42)

modelo_forest = forest.fit(x_train, y_train)

y_pred = modelo_forest.predict(x_validation)
y_pred = np.power(10, y_pred) # devolvemos los valores reales a precio

mean_absolute_error(y_validation, y_pred)

## hiperparametros

el modelo que mejor ha funcionado ha sido catboost asi que voy a estudiar los hiper con este

In [ ]:
x_train

In [ ]:
param_grid = {
    'iterations': [i for i in range(40, 100, 15)],
    'depth': [5, 7, 10, 15],
    'learning_rate': [i for i in np.arange(0.15, 0.31, 0.01)],
    'l2_leaf_reg': [i for i in np.arange(0.5, 21, 0.4)]
}

In [ ]:
model_catboost_1 = CatBoostRegressor(random_state= 42)

In [ ]:
grid_search = RandomizedSearchCV(model_catboost_1, param_grid, cv=5, n_jobs= -1, scoring= 'neg_mean_absolute_error', verbose= 0)
grid_search.fit(x_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
best_catboost = grid_search.best_estimator_

In [ ]:
y_pred = best_catboost.predict(x_validation)
y_pred = np.power(10, y_pred)

mean_absolute_error(y_validation, y_pred)